In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import glob
import requests

# Import State and Local Price deflators

State and local price deflators were downloaded [here](https://research.stlouisfed.org/fred2/series/A829RD3A086NBEA#)

In [2]:
defl = pd.read_excel('/Users/chadsmith/Google Drive/TEL Data/debt_data/state_local_deflators.xls', header=10 )

In [3]:
defl.rename(columns={'A829RD3A086NBEA': 'SLDefl'}, inplace=True)

In [4]:
defl['observation_date'] = defl.observation_date.astype('str')

In [5]:
defl['yr']= defl['observation_date'].str[0:4]

In [6]:
defl.drop('observation_date', inplace=True, axis = 1)

Now I need to convert the index into a multiplier. Since 2009 is the base year, we need the value for 2009.

In [7]:
defl.loc[defl['yr']=='2009']

,SLDefl,yr
81,102.714,2009



The formula for the index will be Base/Comparison

In [8]:
defl['index'] = 102.714/defl['SLDefl']

In [9]:
defl.head()

,SLDefl,yr,index
0,3.968,1929,25.885585
1,3.862,1930,26.596064
2,3.632,1931,28.280286
3,3.258,1932,31.526703
4,3.372,1933,30.460854


Now I need to import the personal income and population data from [here](http://www.bea.gov/regional/downloadzip.cfm)

In [10]:
inc = pd.read_csv('/Users/chadsmith/Google Drive/TEL Data/debt_data/SA1_1929_2014.csv')

In [11]:
inc.head()

,GeoFIPS,GeoName,Region,Table,LineCode,IndustryClassification,Description,1929,1930,1931,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,0,United States,NaN,SA1,1,...,Personal income (thousands of dollars),85126000,76371000,65507000,...,10610320000,11381350000,11995419000,12492705000,12079444000,12459613000,13233436000,13904485000,14064468000,14683147000
1,0,United States,NaN,SA1,2,...,Population (persons) 1/,121769000,123075000,124038000,...,295516599,298379912,301231207,304093966,306771529,309347057,311721632,314112078,316497531,318857056
2,0,United States,NaN,SA1,3,...,Per capita personal income (dollars) 2/,699,621,528,...,35904,38144,39821,41082,39376,40277,42453,44266,44438,46049
3,1000,Alabama,5,SA1,1,...,Personal income (thousands of dollars),842894,697154,583342,...,138019022,146661249,153787754,159993535,157141435,163066901,169030399,173601429,174876574,181908767
4,1000,Alabama,5,SA1,2,...,Population (persons) 1/,2644000,2647000,2649000,...,4569805,4628981,4672840,4718206,4757938,4785822,4801695,4817484,4833996,4849377


In [12]:
droplist = (['Table','IndustryClassification'])

inc.drop(droplist, inplace = True, axis = 1)

In [13]:
inc['GeoFIPS'] = inc['GeoFIPS'].apply(lambda x: x.zfill(5))

In [14]:
df = pd.read_csv('/Users/chadsmith/TELs_debt/data/debt_out.csv')

This is the list of variables (largly costats) that are not required for this analysis

In [15]:
droplist =(['FIPSST','POP_TH18','POP_OV65','TOT_EMP','MFG_EMP','RETL_EMP','T_PUB_SCH',\
            'PUB_SCHL','PVT_SCHL','HSLD_PERS','HSG_UNITS','CH_HS_UNT','PRE_1940','VACANT',\
            'MDHOMEVAL','MED_INC','PC_INC','LANDAREA','GEN_REV','IGR_ST','TAX_REV','PT_REV',\
            'D_GEN_EXP','PC_GEN_EXP','RES_POP','PERS_POVT','SS_PERS','SS_PMT','POVT_PCT','FIPSCO',\
            'ST_MED_H_V','ST_MED_INC','ST_PC_INC','ST_L_GREV','ST_IGR_ST','ST_L_TAX','ST_PT_REV',\
            'ST_L_EXP','ST_P_LEXP','FIPST_N','RATE_L','RATE_L2','MRATE_L','CRATE_L','SRATE_L',\
            'SRATE_L2','MRATE_L2','CRATE_L2','MLEVY_L','CLEVY_L','CLEVY_L2','MLEVY_L2','MLEVY_L3',\
            'SLEVY_L','CLEVY_L3','CLEVY_L4','MLEVY_L4','SLEVY_L2','SLEVY_L3','SLEVY_L4','ASMT_L',\
            'ASMT_L2','ASMT_L3','CREVU_L','MREVU_L','SREVU_L','CGEXP_L','MGEXP_L','SGEXP_L','SGEXP_L2',\
            'MGEXP_L2','CFDISC_L','MFDISC_L','SFDISC_L','HOME_STEAD','HOME_STEAD2','HOME_STEAD3','CB_E',\
            'CB_G','CB_G2','CB_E2','CB_E3','CB_E4','FFDISC_L','TYPE1','TYPE2','BOTH','LIMITS',\
            'TYPE2_Y','SPC_RATE','LEVY_L','REVU_L','GEXP_L','GP_RATE','GP_LEVY','GP_REVU','GP_GEXP',\
            'GP_LMT','SC_LMT','TREND','FOOD_SERV_EMP_PNFARM','PRV_SCHL_KIND','OTH_SERV_EMP_PNFARM',\
            'PRV_SCHL_9_12','MANU_EMP_PNFARM','PUB_SCHL_OV3_M','PRV_SCHL_9_12_F','PRV_SCHL_ELEM_HS',\
            'PUB_SCHL_ELEM_HS','RES_POP1','PUB_SCHL_OV3_F','PRV_SCHL_PREK_F','RETL_EMP_PNFARM',\
            'PROF_SERV_EMP_PNFARM','PRV_SCHL_KIND_M','PRV_SCHL_PREK_M','SUPP_SERV_EMP_PNFARM',\
            'POV_EST_FAM_NUMER','EDUC_SERV_EMP_PNFARM','PRV_SCHL_PREK','PRV_SCHL_KIND_F',\
            'PRV_SCHL_1_8','PRV_SCHL_5_8_M','PUB_SCHL_TOT','PRV_SCHL_1_4_M','HSG_UNITS_ACS',\
            'TOT_AREA','POV_EST_FAM_DENOM','PUB_SCHL_OV3','RES_POP2','PRV_SCHL_9_12_M',\
            'PRV_SCHL_1_4_F','PRV_SCHL_5_8_F','TOT_EMP_PNFARM','RESPOP','DENSITY','POPGROWTH',\
            'PYOUNG','POP65','RESPOP2','PRE1940','PVT_SCH_02','PVT_SCH03_','POVERTY','PC_SSI',\
            'DIVERSITY','EMP_RES','MANU_RES','RETL_RES','SERV_RES'])

In [16]:
df.drop(droplist, inplace = True, axis = 1)

In [27]:
df['FIPS']=df.FIPS.astype('str')
df['Year']=df.Year.astype('str')

In [28]:
df['FIPS'] = df['FIPS'].apply(lambda x: x.zfill(5))

This establishes the list of dollar values that need to be deflated

In [29]:
dollar_list =(['GO','RV','GO_City, Town Vlg','GO_Co-op Utility',\
               'GO_College or Univ','GO_County/Parish','GO_Direct Issuer',\
               'GO_District','GO_Indian Tribe','GO_Local Authority',\
               'GO_State Authority','GO_State/Province','RV_City, Town Vlg',\
               'RV_Co-op Utility','RV_College or Univ','RV_County/Parish',\
               'RV_Direct Issuer','RV_District','RV_Indian Tribe','RV_Local Authority',\
               'RV_State Authority','RV_State/Province','GO_Development','GO_Education',\
               'GO_Electric Power','GO_Environmental Facilities','GO_General Purpose',\
               'GO_Healthcare','GO_Housing','GO_Public Facilities','GO_Transportation',\
               'GO_Utilities','RV_Development','RV_Education','RV_Electric Power',\
               'RV_Environmental Facilities','RV_General Purpose','RV_Healthcare',\
               'RV_Housing','RV_Public Facilities','RV_Transportation','RV_Utilities'])

In [30]:
inc.head()

,GeoFIPS,GeoName,Region,LineCode,Description,1929,1930,1931,1932,1933,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,00000,United States,NaN,1,Personal income (thousands of dollars),85126000,76371000,65507000,50134000,47085000,...,10610320000,11381350000,11995419000,12492705000,12079444000,12459613000,13233436000,13904485000,14064468000,14683147000
1,00000,United States,NaN,2,Population (persons) 1/,121769000,123075000,124038000,124839000,125580000,...,295516599,298379912,301231207,304093966,306771529,309347057,311721632,314112078,316497531,318857056
2,00000,United States,NaN,3,Per capita personal income (dollars) 2/,699,621,528,402,375,...,35904,38144,39821,41082,39376,40277,42453,44266,44438,46049
3,01000,Alabama,5,1,Personal income (thousands of dollars),842894,697154,583342,421538,435290,...,138019022,146661249,153787754,159993535,157141435,163066901,169030399,173601429,174876574,181908767
4,01000,Alabama,5,2,Population (persons) 1/,2644000,2647000,2649000,2653000,2661000,...,4569805,4628981,4672840,4718206,4757938,4785822,4801695,4817484,4833996,4849377


I need to transpose this so that each year is a row

In [31]:
pce.drop('PCECTPI', inplace = True, axis = 1)

ValueError: labels ['PCECTPI'] not contained in axis

In [35]:
df = pd.merge(pce, df, left_on = 'yr', right_on = 'Year', how = 'right')

In [36]:
df.drop('yr',inplace = True, axis = 1)

In [46]:
for i in dollar_list:
    df[i] = df[i] * df['index']

In [47]:
df.head()

,index,Year,FIPS,GO,RV,"GO_City, Town Vlg",GO_Co-op Utility,GO_College or Univ,GO_County/Parish,GO_Direct Issuer,...,RV_Developmentadj,RV_Educationadj,RV_Electric Poweradj,RV_Environmental Facilitiesadj,RV_General Purposeadj,RV_Healthcareadj,RV_Housingadj,RV_Public Facilitiesadj,RV_Transportationadj,RV_Utilitiesadj
0,1.794056,1984,01000,0.000000,1431.570475,0,0,0,0.000000,0,...,10.764335,0.000000,0.000000,0,0.00000,464.875759,221.964181,0,733.9662,0.000000
1,1.794056,1984,01001,0.000000,2.915341,0,0,0,0.000000,0,...,0.000000,0.000000,0.000000,0,0.00000,0.000000,0.000000,0,0.0000,2.915341
2,1.794056,1984,01003,11.392255,41.855324,0,0,0,11.392255,0,...,3.408706,21.528671,11.472987,0,0.00000,0.000000,0.000000,0,0.0000,5.444960
3,1.794056,1984,01007,0.000000,0.717622,0,0,0,0.000000,0,...,0.717622,0.000000,0.000000,0,0.00000,0.000000,0.000000,0,0.0000,0.000000
4,1.794056,1984,01021,1.794056,2.556530,0,0,0,1.794056,0,...,0.000000,0.000000,0.000000,0,2.55653,0.000000,0.000000,0,0.0000,0.000000


We will be using the BEA region definitions that are found [here](http://www.bea.gov/regional/docs/regions.cfm). This file was added to the data file and labeled region.